In [1]:
import os
import shutil
from tqdm import tqdm

# Ruta base donde se encuentran los datos
base_path = r'C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas'

def find_movement_folders():
    """
    Busca todas las carpetas llamadas 'Movimiento' en la estructura de directorios
    
    Returns:
        list: Lista de tuplas (expresión, muestra, ruta_completa_carpeta)
    """
    movement_folders = []
    
    # Recorrer todas las carpetas de expresiones
    for expression in os.listdir(base_path):
        expression_path = os.path.join(base_path, expression)
        
        if not os.path.isdir(expression_path):
            continue
            
        # Recorrer las carpetas de muestras
        try:
            for muestra_folder in os.listdir(expression_path):
                muestra_path = os.path.join(expression_path, muestra_folder)
                
                if not os.path.isdir(muestra_path) or not muestra_folder.startswith('muestra_'):
                    continue
                
                # Buscar la carpeta 'Movimiento' dentro de la muestra
                movement_path = os.path.join(muestra_path, 'keyframes_2')
                if os.path.exists(movement_path) and os.path.isdir(movement_path):
                    # Contar archivos en la carpeta
                    num_files = len([f for f in os.listdir(movement_path) if os.path.isfile(os.path.join(movement_path, f))])
                    # Obtener tamaño total de la carpeta
                    folder_size = get_folder_size(movement_path)
                    movement_folders.append((expression, muestra_folder, movement_path, num_files, folder_size))
        except Exception as e:
            print(f"Error al explorar {expression_path}: {e}")
    
    return movement_folders

def get_folder_size(folder_path):
    """
    Calcula el tamaño total de una carpeta en bytes
    
    Args:
        folder_path: Ruta de la carpeta
    
    Returns:
        int: Tamaño de la carpeta en bytes
    """
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # Saltarse si es un enlace simbólico
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
    return total_size

def display_movement_folders(folders):
    """
    Muestra la lista de carpetas 'Movimiento' encontradas con detalles
    
    Args:
        folders: Lista de tuplas (expresión, muestra, ruta_completa, num_archivos, tamaño)
    """
    if not folders:
        print("❌ No se encontraron carpetas 'Movimiento'.")
        return
    
    print(f"📊 Carpetas 'Movimiento' encontradas ({len(folders)}):\n")
    
    for i, (expression, muestra, folder_path, num_files, size) in enumerate(folders, 1):
        size_mb = size / (1024 * 1024)  # Convertir a MB
        
        print(f"{i}. {expression}/{muestra}/Movimiento")
        print(f"   - Ruta: {folder_path}")
        print(f"   - Archivos: {num_files}")
        print(f"   - Tamaño: {size_mb:.2f} MB\n")

def delete_all_movement_folders(folders):
    """
    Elimina todas las carpetas 'Movimiento' de la lista
    
    Args:
        folders: Lista de tuplas (expresión, muestra, ruta_completa, num_archivos, tamaño)
    
    Returns:
        int: Número de carpetas eliminadas
    """
    if not folders:
        print("❌ No hay carpetas 'Movimiento' para eliminar.")
        return 0
    
    deleted_count = 0
    
    for expression, muestra, folder_path, _, _ in tqdm(folders, desc="Eliminando carpetas"):
        try:
            shutil.rmtree(folder_path)
            deleted_count += 1
        except Exception as e:
            print(f"⚠️ No se pudo eliminar {folder_path}: {e}")
    
    return deleted_count

def delete_movement_folders_by_expression(folders, target_expression):
    """
    Elimina las carpetas 'Movimiento' de una expresión específica
    
    Args:
        folders: Lista de tuplas (expresión, muestra, ruta_completa, num_archivos, tamaño)
        target_expression: Nombre de la expresión cuyas carpetas se quieren eliminar
    
    Returns:
        int: Número de carpetas eliminadas
    """
    if not folders:
        print("❌ No hay carpetas 'Movimiento' para eliminar.")
        return 0
    
    # Filtrar carpetas por expresión
    folders_to_delete = [f for f in folders if f[0].lower() == target_expression.lower()]
    
    if not folders_to_delete:
        print(f"❌ No se encontraron carpetas 'Movimiento' para la expresión '{target_expression}'.")
        return 0
    
    # Mostrar carpetas que se van a eliminar
    print(f"Se eliminarán {len(folders_to_delete)} carpetas 'Movimiento' de la expresión '{target_expression}':")
    for i, (_, muestra, folder_path, num_files, size) in enumerate(folders_to_delete, 1):
        size_mb = size / (1024 * 1024)
        print(f"  {i}. {muestra}/Movimiento - {num_files} archivos, {size_mb:.2f} MB")
    
    # Confirmar eliminación
    confirm = input("\n¿Estás seguro de que deseas eliminar estas carpetas? (s/n): ")
    if confirm.lower() != 's':
        print("❌ Operación cancelada.")
        return 0
    
    deleted_count = 0
    
    for _, _, folder_path, _, _ in tqdm(folders_to_delete, desc=f"Eliminando carpetas de '{target_expression}'"):
        try:
            shutil.rmtree(folder_path)
            deleted_count += 1
        except Exception as e:
            print(f"⚠️ No se pudo eliminar {folder_path}: {e}")
    
    return deleted_count

def delete_specific_movement_folders(folders):
    """
    Permite al usuario seleccionar carpetas 'Movimiento' específicas para eliminar
    
    Args:
        folders: Lista de tuplas (expresión, muestra, ruta_completa, num_archivos, tamaño)
    
    Returns:
        int: Número de carpetas eliminadas
    """
    if not folders:
        print("❌ No hay carpetas 'Movimiento' para eliminar.")
        return 0
    
    # Mostrar carpetas numeradas
    print("Selecciona las carpetas 'Movimiento' que deseas eliminar (ingresa los números separados por comas):\n")
    for i, (expression, muestra, folder_path, num_files, size) in enumerate(folders, 1):
        size_mb = size / (1024 * 1024)
        print(f"{i}. {expression}/{muestra}/Movimiento - {num_files} archivos, {size_mb:.2f} MB")
    
    # Solicitar selección
    selection = input("\nNúmeros de carpetas a eliminar (ej: 1,3,5-7): ")
    
    # Procesar selección
    selected_indices = set()
    
    for part in selection.split(','):
        if '-' in part:
            # Rango (ej: 5-7)
            try:
                start, end = map(int, part.split('-'))
                if 1 <= start <= len(folders) and 1 <= end <= len(folders):
                    selected_indices.update(range(start, end + 1))
            except ValueError:
                print(f"⚠️ Rango inválido: {part}")
        else:
            # Número individual
            try:
                index = int(part)
                if 1 <= index <= len(folders):
                    selected_indices.add(index)
            except ValueError:
                print(f"⚠️ Número inválido: {part}")
    
    if not selected_indices:
        print("❌ No se seleccionaron carpetas válidas para eliminar.")
        return 0
    
    # Convertir a lista y ordenar
    selected_indices = sorted(list(selected_indices))
    
    # Confirmar eliminación
    print(f"\nSe eliminarán {len(selected_indices)} carpetas:")
    for idx in selected_indices:
        expression, muestra, folder_path, num_files, size = folders[idx - 1]
        size_mb = size / (1024 * 1024)
        print(f"  - {expression}/{muestra}/Movimiento - {num_files} archivos, {size_mb:.2f} MB")
    
    confirm = input("\n¿Estás seguro de que deseas eliminar estas carpetas? (s/n): ")
    if confirm.lower() != 's':
        print("❌ Operación cancelada.")
        return 0
    
    # Eliminar carpetas seleccionadas
    deleted_count = 0
    
    for idx in selected_indices:
        expression, muestra, folder_path, _, _ = folders[idx - 1]
        try:
            shutil.rmtree(folder_path)
            deleted_count += 1
            print(f"✅ Eliminada: {expression}/{muestra}/Movimiento")
        except Exception as e:
            print(f"⚠️ No se pudo eliminar {folder_path}: {e}")
    
    return deleted_count

# Menú principal
if __name__ == "__main__":
    print("🔍 Buscando carpetas 'Movimiento'...")
    movement_folders = find_movement_folders()
    
    if not movement_folders:
        print("❌ No se encontraron carpetas 'Movimiento' en la estructura de directorios.")
        exit()
    
    while True:
        print("\n===== ELIMINADOR DE CARPETAS 'MOVIMIENTO' =====")
        print(f"Carpetas encontradas: {len(movement_folders)}")
        print("\n¿Qué acción deseas realizar?")
        print("1. Ver la lista de carpetas 'Movimiento'")
        print("2. Eliminar todas las carpetas 'Movimiento'")
        print("3. Eliminar carpetas 'Movimiento' de una expresión específica")
        print("4. Eliminar carpetas 'Movimiento' específicas")
        print("5. Salir")
        
        choice = input("\nIngresa tu opción (1-5): ")
        
        if choice == "1":
            display_movement_folders(movement_folders)
        
        elif choice == "2":
            confirm = input(f"¿Estás seguro de que deseas eliminar TODAS ({len(movement_folders)}) las carpetas 'Movimiento'? (s/n): ")
            if confirm.lower() == 's':
                deleted = delete_all_movement_folders(movement_folders)
                print(f"\n✅ {deleted} carpetas 'Movimiento' fueron eliminadas.")
                if deleted > 0:
                    # Actualizar la lista de carpetas
                    movement_folders = find_movement_folders()
            else:
                print("❌ Operación cancelada.")
        
        elif choice == "3":
            # Mostrar expresiones disponibles
            expressions = sorted(set(f[0] for f in movement_folders))
            print("\nExpresiones disponibles:")
            for i, exp in enumerate(expressions, 1):
                count = sum(1 for f in movement_folders if f[0] == exp)
                print(f"{i}. {exp} ({count} carpetas 'Movimiento')")
            
            exp_choice = input("\nIngresa el nombre de la expresión: ")
            
            deleted = delete_movement_folders_by_expression(movement_folders, exp_choice)
            if deleted > 0:
                print(f"\n✅ {deleted} carpetas 'Movimiento' fueron eliminadas.")
                # Actualizar la lista de carpetas
                movement_folders = find_movement_folders()
        
        elif choice == "4":
            deleted = delete_specific_movement_folders(movement_folders)
            if deleted > 0:
                print(f"\n✅ {deleted} carpetas 'Movimiento' fueron eliminadas.")
                # Actualizar la lista de carpetas
                movement_folders = find_movement_folders()
        
        elif choice == "5":
            print("👋 ¡Hasta luego!")
            break
        
        else:
            print("❌ Opción inválida. Por favor, intenta de nuevo.")

🔍 Buscando carpetas 'Movimiento'...

===== ELIMINADOR DE CARPETAS 'MOVIMIENTO' =====
Carpetas encontradas: 83

¿Qué acción deseas realizar?
1. Ver la lista de carpetas 'Movimiento'
2. Eliminar todas las carpetas 'Movimiento'
3. Eliminar carpetas 'Movimiento' de una expresión específica
4. Eliminar carpetas 'Movimiento' específicas
5. Salir
📊 Carpetas 'Movimiento' encontradas (83):

1. A ver/muestra_1/Movimiento
   - Ruta: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_1\keyframes_2
   - Archivos: 37
   - Tamaño: 3.29 MB

2. A ver/muestra_2/Movimiento
   - Ruta: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_2\keyframes_2
   - Archivos: 33
   - Tamaño: 2.91 MB

3. A ver/muestra_3/Movimiento
   - Ruta: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_3\keyframes_2
   - Archivos: 25
   - Ta

Eliminando carpetas: 100%|██████████| 83/83 [00:00<00:00, 84.50it/s] 



✅ 83 carpetas 'Movimiento' fueron eliminadas.

===== ELIMINADOR DE CARPETAS 'MOVIMIENTO' =====
Carpetas encontradas: 0

¿Qué acción deseas realizar?
1. Ver la lista de carpetas 'Movimiento'
2. Eliminar todas las carpetas 'Movimiento'
3. Eliminar carpetas 'Movimiento' de una expresión específica
4. Eliminar carpetas 'Movimiento' específicas
5. Salir
👋 ¡Hasta luego!


In [1]:
import os
import shutil
from tqdm import tqdm

def list_cara_folders(base_path):
    """
    Recorre la estructura de directorios desde base_path
    y lista todas las carpetas llamadas 'CARA'
    
    Args:
        base_path: Ruta base desde donde iniciar la búsqueda
        
    Returns:
        list: Lista de rutas de todas las carpetas CARA encontradas
    """
    # Lista para almacenar las rutas de todas las carpetas CARA encontradas
    cara_folders = []
    
    print(f"🔍 Buscando carpetas 'CARA' en: {base_path}")
    
    # Recorrer toda la estructura de directorios
    for root, dirs, files in os.walk(base_path):
        # Verificar si hay una carpeta llamada 'CARA' en este directorio
        if 'CARA' in dirs:
            cara_path = os.path.join(root, 'CARA')
            cara_folders.append(cara_path)
        elif 'cara' in dirs:
            cara_path = os.path.join(root, 'cara')
            cara_folders.append(cara_path)
    
    return cara_folders

def remove_cara_folders_without_confirmation(base_path):
    """
    Elimina directamente todas las carpetas CARA sin pedir confirmación
    
    Args:
        base_path: Ruta base desde donde iniciar la búsqueda
        
    Returns:
        int: Número de carpetas eliminadas
    """
    # Obtener lista de carpetas CARA
    cara_folders = list_cara_folders(base_path)
    
    # Mostrar todas las carpetas encontradas
    if cara_folders:
        print(f"\n✅ Se encontraron {len(cara_folders)} carpetas 'CARA':")
        for path in cara_folders:
            print(f"  - {path}")
        
        # Eliminar cada carpeta con una barra de progreso
        removed = 0
        for path in tqdm(cara_folders, desc="Eliminando carpetas CARA"):
            try:
                shutil.rmtree(path)
                removed += 1
            except Exception as e:
                print(f"❌ Error eliminando {path}: {e}")
        
        print(f"\n✅ Se eliminaron {removed}/{len(cara_folders)} carpetas CARA")
        return removed
    else:
        print("\n❓ No se encontraron carpetas 'CARA' en la estructura de directorios")
        return 0

# Función principal
if __name__ == "__main__":
    print("===== ELIMINADOR DE CARPETAS CARA =====")
    print("Este script eliminará todas las carpetas CARA en la estructura de directorios")
    print("==========================================\n")
    
    # Usar directamente la ruta específica para SUJETO 2
    specific_path = r"C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas"
    
    if os.path.exists(specific_path):
        remove_cara_folders_without_confirmation(specific_path)
    else:
        print(f"❌ La ruta {specific_path} no existe")

===== ELIMINADOR DE CARPETAS CARA =====
Este script eliminará todas las carpetas CARA en la estructura de directorios

🔍 Buscando carpetas 'CARA' en: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas

✅ Se encontraron 83 carpetas 'CARA':
  - C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_1\CARA
  - C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_2\CARA
  - C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_3\CARA
  - C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_4\CARA
  - C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_5\CARA
  - C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SU

Eliminando carpetas CARA: 100%|██████████| 83/83 [00:01<00:00, 52.44it/s]


✅ Se eliminaron 83/83 carpetas CARA


In [2]:
import os
import shutil
from tqdm import tqdm

# Ruta base donde se encuentran los datos
base_path = r'C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas'

def find_pattern_files():
    """
    Busca todos los archivos que tienen formato de muestra pero que NO comienzan con "video"
    
    Returns:
        list: Lista de tuplas (ruta_completa_archivo, nombre_archivo, tamaño)
    """
    pattern_files = []
    
    # Recorrer toda la estructura de directorios
    for root, _, files in os.walk(base_path):
        for file in files:
            # Verificar si el archivo tiene el formato de muestra (contiene "muestra_")
            if "muestra_" in file and not file.startswith("video"):
                file_path = os.path.join(root, file)
                file_size = os.path.getsize(file_path)
                pattern_files.append((file_path, file, file_size))
    
    return pattern_files

def display_pattern_files(files):
    """
    Muestra la lista de archivos encontrados con detalles
    
    Args:
        files: Lista de tuplas (ruta_completa_archivo, nombre_archivo, tamaño)
    """
    if not files:
        print("❌ No se encontraron archivos que contengan 'muestra_' y no comiencen con 'video'.")
        return
    
    print(f"📊 Archivos encontrados ({len(files)}):\n")
    
    for i, (file_path, file_name, size) in enumerate(files, 1):
        size_kb = size / 1024  # Convertir a KB
        
        print(f"{i}. {file_name}")
        print(f"   - Ruta: {file_path}")
        print(f"   - Tamaño: {size_kb:.2f} KB\n")

def delete_all_pattern_files(files):
    """
    Elimina todos los archivos de la lista
    
    Args:
        files: Lista de tuplas (ruta_completa_archivo, nombre_archivo, tamaño)
    
    Returns:
        int: Número de archivos eliminados
    """
    if not files:
        print("❌ No hay archivos para eliminar.")
        return 0
    
    deleted_count = 0
    
    for file_path, _, _ in tqdm(files, desc="Eliminando archivos"):
        try:
            os.remove(file_path)
            deleted_count += 1
        except Exception as e:
            print(f"⚠️ No se pudo eliminar {file_path}: {e}")
    
    return deleted_count



def delete_specific_pattern_files(files):
    """
    Permite al usuario seleccionar archivos específicos para eliminar
    
    Args:
        files: Lista de tuplas (ruta_completa_archivo, nombre_archivo, tamaño)
    
    Returns:
        int: Número de archivos eliminados
    """
    if not files:
        print("❌ No hay archivos para eliminar.")
        return 0
    
    # Mostrar archivos numerados
    print("Selecciona los archivos que deseas eliminar (ingresa los números separados por comas):\n")
    for i, (file_path, file_name, size) in enumerate(files, 1):
        size_kb = size / 1024
        print(f"{i}. {file_name} - {size_kb:.2f} KB")
    
    # Solicitar selección
    selection = input("\nNúmeros de archivos a eliminar (ej: 1,3,5-7): ")
    
    # Procesar selección
    selected_indices = set()
    
    for part in selection.split(','):
        if '-' in part:
            # Rango (ej: 5-7)
            try:
                start, end = map(int, part.split('-'))
                if 1 <= start <= len(files) and 1 <= end <= len(files):
                    selected_indices.update(range(start, end + 1))
            except ValueError:
                print(f"⚠️ Rango inválido: {part}")
        else:
            # Número individual
            try:
                index = int(part)
                if 1 <= index <= len(files):
                    selected_indices.add(index)
            except ValueError:
                print(f"⚠️ Número inválido: {part}")
    
    if not selected_indices:
        print("❌ No se seleccionaron archivos válidos para eliminar.")
        return 0
    
    # Convertir a lista y ordenar
    selected_indices = sorted(list(selected_indices))
    
    # Confirmar eliminación
    print(f"\nSe eliminarán {len(selected_indices)} archivos:")
    for idx in selected_indices:
        file_path, file_name, size = files[idx - 1]
        size_kb = size / 1024
        print(f"  - {file_name} - {size_kb:.2f} KB")
    
    confirm = input("\n¿Estás seguro de que deseas eliminar estos archivos? (s/n): ")
    if confirm.lower() != 's':
        print("❌ Operación cancelada.")
        return 0
    
    # Eliminar archivos seleccionados
    deleted_count = 0
    
    for idx in selected_indices:
        file_path, file_name, _ = files[idx - 1]
        try:
            os.remove(file_path)
            deleted_count += 1
            print(f"✅ Eliminado: {file_name}")
        except Exception as e:
            print(f"⚠️ No se pudo eliminar {file_path}: {e}")
    
    return deleted_count

# Menú principal
if __name__ == "__main__":
    print("🔍 Buscando archivos con formato [expresion]_muestra_X...")
    pattern_files = find_pattern_files()
    
    if not pattern_files:
        print("❌ No se encontraron archivos con el formato [expresion]_muestra_X en la estructura de directorios.")
        exit()
    
    while True:
        print("\n===== ELIMINADOR DE ARCHIVOS CON PATRÓN [expresion]_muestra_X =====")
        print(f"Archivos encontrados: {len(pattern_files)}")
        print("\n¿Qué acción deseas realizar?")
        print("1. Ver la lista de archivos encontrados")
        print("2. Eliminar todos los archivos encontrados")
        print("3. Eliminar archivos de una expresión específica")
        print("4. Eliminar archivos específicos")
        print("5. Salir")
        
        choice = input("\nIngresa tu opción (1-4): ")
        
        if choice == "1":
            display_pattern_files(pattern_files)
        
        elif choice == "2":
            confirm = input(f"¿Estás seguro de que deseas eliminar TODOS ({len(pattern_files)}) los archivos encontrados? (s/n): ")
            if confirm.lower() == 's':
                deleted = delete_all_pattern_files(pattern_files)
                print(f"\n✅ {deleted} archivos fueron eliminados.")
                if deleted > 0:
                    # Actualizar la lista de archivos
                    pattern_files = find_pattern_files()
            else:
                print("❌ Operación cancelada.")
        
        elif choice == "3":
            deleted = delete_specific_pattern_files(pattern_files)
            if deleted > 0:
                print(f"\n✅ {deleted} archivos fueron eliminados.")
                # Actualizar la lista de archivos
                pattern_files = find_pattern_files()
        
        elif choice == "4":
            print("👋 ¡Hasta luego!")
            break
        
        else:
            print("❌ Opción inválida. Por favor, intenta de nuevo.")

🔍 Buscando archivos con formato [expresion]_muestra_X...

===== ELIMINADOR DE ARCHIVOS CON PATRÓN [expresion]_muestra_X =====
Archivos encontrados: 82

¿Qué acción deseas realizar?
1. Ver la lista de archivos encontrados
2. Eliminar todos los archivos encontrados
3. Eliminar archivos de una expresión específica
4. Eliminar archivos específicos
5. Salir
📊 Archivos encontrados (82):

1. a_ver_muestra_1.mp4
   - Ruta: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_1\a_ver_muestra_1.mp4
   - Tamaño: 715.85 KB

2. a_ver_muestra_3.mp4
   - Ruta: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_3\a_ver_muestra_3.mp4
   - Tamaño: 632.79 KB

3. a_ver_muestra_4.mp4
   - Ruta: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_4\a_ver_muestra_4.mp4
   - Tamaño: 733.65 KB

4. a_ver_muestra_5.mp4
   - Ru

Eliminando archivos: 100%|██████████| 82/82 [00:00<00:00, 4920.50it/s]



✅ 82 archivos fueron eliminados.

===== ELIMINADOR DE ARCHIVOS CON PATRÓN [expresion]_muestra_X =====
Archivos encontrados: 0

¿Qué acción deseas realizar?
1. Ver la lista de archivos encontrados
2. Eliminar todos los archivos encontrados
3. Eliminar archivos de una expresión específica
4. Eliminar archivos específicos
5. Salir
❌ Opción inválida. Por favor, intenta de nuevo.

===== ELIMINADOR DE ARCHIVOS CON PATRÓN [expresion]_muestra_X =====
Archivos encontrados: 0

¿Qué acción deseas realizar?
1. Ver la lista de archivos encontrados
2. Eliminar todos los archivos encontrados
3. Eliminar archivos de una expresión específica
4. Eliminar archivos específicos
5. Salir
❌ Opción inválida. Por favor, intenta de nuevo.

===== ELIMINADOR DE ARCHIVOS CON PATRÓN [expresion]_muestra_X =====
Archivos encontrados: 0

¿Qué acción deseas realizar?
1. Ver la lista de archivos encontrados
2. Eliminar todos los archivos encontrados
3. Eliminar archivos de una expresión específica
4. Eliminar archivos 